In [ ]:
#Dependencies: Gym, Numpy, Random
#Installing Gym in Linux using conda https://anaconda.org/akode/gym

#Install the dependencies using
!python -m pip install pyvirtualdisplay
!apt-get install xvfb
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
import gym
import sys
import itertools
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline   #This will lead to static images of your plot embedded in the notebook